In [41]:
import MetaTrader5 as mt
import datetime

In [42]:
mt.initialize()

True

In [43]:
from datetime import datetime
import pandas as pd

symbol = "TSLA"
rates = mt.copy_rates_from_pos(symbol, mt.TIMEFRAME_H1, 0, 1000)

data = pd.DataFrame(rates)
data['time'] = pd.to_datetime(data['time'], unit='s')
data.set_index('time', inplace=True)
print(data.tail())


                       open    high     low   close  tick_volume  spread  \
time                                                                       
2025-10-07 19:00:00  441.08  446.74  440.14  446.12        24644       1   
2025-10-07 20:00:00  446.15  450.20  445.73  447.28        21168       1   
2025-10-07 21:00:00  447.36  450.01  437.74  440.83        40640       1   
2025-10-07 22:00:00  440.88  442.22  432.45  432.87        58848       1   
2025-10-07 23:00:00  433.08  433.09  433.08  433.09         1030       1   

                     real_volume  
time                              
2025-10-07 19:00:00      5850384  
2025-10-07 20:00:00      5019083  
2025-10-07 21:00:00      9305676  
2025-10-07 22:00:00     12220010  
2025-10-07 23:00:00      3436702  


In [44]:
data['EMA30'] = data['close'].ewm(span=30, adjust=False).mean()
data['EMA70'] = data['close'].ewm(span=70, adjust=False).mean()


In [45]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
open,1000.0,3.262082e+02,5.467878e+01,223.000000,2.899325e+02,3.243500e+02,3.465850e+02,4.705400e+02
high,1000.0,3.286054e+02,5.443915e+01,225.140000,2.934525e+02,3.263150e+02,3.484800e+02,4.707500e+02
low,1000.0,3.239553e+02,5.464794e+01,214.280000,2.869150e+02,3.221500e+02,3.449000e+02,4.594100e+02
close,1000.0,3.262834e+02,5.448941e+01,221.640000,2.911175e+02,3.245000e+02,3.464950e+02,4.601100e+02
tick_volume,1000.0,3.383339e+04,2.075205e+04,292.000000,2.160575e+04,3.131500e+04,4.332175e+04,2.107140e+05
spread,1000.0,3.073300e+01,3.786206e+02,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,7.268000e+03
real_volume,1000.0,7.979569e+06,4.224633e+06,283419.000000,5.098319e+06,7.136053e+06,1.000190e+07,4.721684e+07
EMA30,1000.0,3.232896e+02,5.232023e+01,235.025878,2.822136e+02,3.227673e+02,3.418913e+02,4.470913e+02
EMA70,1000.0,3.192768e+02,4.941889e+01,235.303051,2.760670e+02,3.226169e+02,3.377351e+02,4.389700e+02


### Detect Crossover

In [46]:
data['Signal'] = 0
data.loc[data['EMA30'] > data['EMA70'], 'Signal'] = 1   # Buy
data.loc[data['EMA30'] < data['EMA70'], 'Signal'] = -1  # Sell

# Detect crossover points
data['Crossover'] = data['Signal'].diff()
buy_signals = data[data['Crossover'] == 2]   # EMA30 crosses above EMA70
sell_signals = data[data['Crossover'] == -2] # EMA30 crosses below EMA70


### Define Entry, Stop Loss, and Take Profit

In [47]:
capital = 10000
risk_pct = 0.03  # 3%

# Example for one trade
entry_price = data['close'].iloc[-1]
stop_distance = entry_price * risk_pct / 100  # simplified, you can link to pip distance
stop_loss = entry_price - stop_distance
take_profit = entry_price + (2 * stop_distance)  # Risk:Reward = 1:2


In [54]:
symbol = "TSLA"
info = mt.symbol_info(symbol)
print(info)

SymbolInfo(custom=False, chart_mode=1, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=619, volumehigh=263000, volumelow=100, time=1759892400, digits=2, spread=10, spread_float=True, ticks_bookdepth=0, trade_calc_mode=32, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=5, trade_freeze_level=0, trade_exemode=3, swap_mode=0, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=1, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=436.95, bidhigh=0.0, bidlow=0.0, ask=437.05, askhigh=0.0, asklow=0.0, last=433.09, lasthigh=452.68, lastlow=432.45, volume_real=619.0, volumehigh_real=263000.0, volumelow_real=100.0, option_strike=0.0, point=0.01, trade_tick_value=0.01, trade_tick_value_profit=0.01, trade_tick_value_loss=0.01, trade_tick_size=0.01, trade_contract_size=1.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=1.0, volume_min=1.0, volume_max=1000000.0

In [55]:
lot = 10.00

request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt.ORDER_TYPE_BUY,
    "price": mt.symbol_info_tick(symbol).ask,
    "sl": stop_loss,
    "tp": take_profit,
    "deviation": 20,
    "magic": 42,
    "comment": "EMA Crossover",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_RETURN,
}

result = mt.order_send(request)
print(result)


OrderSendResult(retcode=10016, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid stops', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=42, order=0, symbol='TSLA', volume=10.0, price=437.05, stoplimit=0.0, sl=432.96007299999997, tp=433.349854, deviation=20, type=0, type_filling=2, type_time=0, expiration=0, comment='EMA Crossover', position=0, position_by=0))
